In [1]:
import os
import pandas as pd
import h5py

%matplotlib inline

from IPython.display import display, HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
def apply_labels(df,table_type):
    '''
    Replace values with human readable lablels.
    '''
    df_label = labels[labels['table'] == table_type]
    for field in df_label['field'].unique():
        newdf = df_label[df_label['field'] == field]
        local_series = pd.Series(newdf['text'].values, index=newdf['value'])
        df[field] = df[field].map(local_series)
    
    return df

In [3]:
# Load data from daysim_outputs.h5, apply labels
daysim = h5py.File(r'../../../outputs/daysim/daysim_outputs.h5', 'r')

trip = pd.DataFrame()
for col in daysim['Trip'].keys():
    trip[col] = daysim['Trip'][col][:]
    
person = pd.DataFrame()
for col in daysim['Person'].keys():
    person[col] = daysim['Person'][col][:]
    
hh = pd.DataFrame()
for col in daysim['Household'].keys():
    hh[col] = daysim['Household'][col][:]
    
# Add labels
labels = pd.read_csv(r'../../../scripts/summarize/inputs/calibration/variable_labels.csv')

trip = apply_labels(trip,'Trip')
person = apply_labels(person,'Person')
hh = apply_labels(hh,'Household')

In [4]:

special_taz = pd.read_csv(r'../../../scripts/summarize/inputs/special_needs_taz.csv')
rgc_taz = pd.read_csv(r'../../../scripts/summarize/inputs/rgc_taz.csv')

In [5]:
# Join RGC, low income, and minority data, based on home location

trip_hh = pd.merge(trip,hh,on='hhno',how='left')

trip_hh = pd.merge(trip_hh, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
trip_hh = pd.merge(trip_hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [6]:
trip_hh = trip_hh[trip_hh['mode']!='School Bus']

## Mode Share
Based on home location

In [7]:
def mode_share(trip_hh):
    """
    Produce trip mode share for low income, minority, and regional populations
    """
    # Regional total
    df_reg = pd.DataFrame(
        trip_hh.groupby('mode').sum()['trexpfac']/trip_hh['trexpfac'].sum())
    df_reg.columns = ['Region']
    df_reg = df_reg.reset_index()
    
    # Low Income
    df = trip_hh[['trexpfac','Low Income','mode']].groupby(['Low Income','mode']).sum().reset_index()
    tot_trips = pd.DataFrame(df.groupby('Low Income').sum()['trexpfac']).reset_index()
    df = pd.merge(df,tot_trips,on='Low Income', suffixes=['_mode','_total'])
    df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

    df_inc = df.pivot(index='Low Income', columns='mode', values='mode_share')
    df_inc.index = ['Other','People of Lower Income']
    
    # Minority
    df = trip_hh[['trexpfac','Minority','mode']].groupby(['Minority','mode']).sum().reset_index()
    tot_trips = pd.DataFrame(df.groupby('Minority').sum()['trexpfac']).reset_index()
    df = pd.merge(df,tot_trips,on='Minority', suffixes=['_mode','_total'])
    df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])

    df_race = df.pivot(index='Minority', columns='mode', values='mode_share')
    df_race.index = ['Other','People of Color']
    
    # Merge all dataframes
    df = pd.merge(pd.DataFrame(df_race.loc['People of Color']).reset_index(),
         pd.DataFrame(df_inc.loc['People of Lower Income']).reset_index())
    df = pd.merge(df_reg, df)
    df.index = df['mode']
    df = df.drop('mode', axis=1)
    return df

### All Trips

In [8]:
# pd.options.display.float_format = '{:.2f}'.format
df = mode_share(trip_hh)
df

,Region,People of Color,People of Lower Income
mode,,,
Bike,0.030425,0.030331,0.041908
HOV2,0.224661,0.228417,0.212250
HOV3+,0.151485,0.159993,0.143658
SOV,0.394553,0.373800,0.333642
Transit,0.050920,0.060459,0.064782
Walk,0.147955,0.147001,0.203760


### Commute Trips

In [9]:
commute_trips = trip_hh[trip_hh['dpurp'] == 'Work']
commute_trips = commute_trips[commute_trips['mode'] != 'School Bus']
df = mode_share(commute_trips)
df

,Region,People of Color,People of Lower Income
mode,,,
Bike,0.042372,0.045623,0.060697
HOV2,0.094214,0.097577,0.091244
HOV3+,0.043656,0.045809,0.041720
SOV,0.592935,0.573406,0.513521
Transit,0.095800,0.105684,0.112562
Walk,0.131022,0.131902,0.180256


### Access to Frequent Transit

In [10]:
#df = pd.read_csv(r'../../../outputs/transit/freq_transit_access.csv')
# Join parcel data (transit access) to household recods
#df = pd.merge(hh[['hhparcel','hhsize','hhtaz']],df[['PARCELID','dist_frequent']],how='left',left_on='hhparcel',right_on='PARCELID')

# join to geography fiels
#df = pd.merge(df, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
#df = pd.merge(df, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

IOError: File ../../../outputs/transit/freq_transit_access.csv does not exist

In [ ]:
# Only include househodls on parcels within 1/2 mile of frequent transit
#max_dist = 0.5
#df_freq = df[df['dist_frequent'] <= max_dist]

In [ ]:
# Calculate percent of people within that range

In [ ]:
#df_lowinc = pd.DataFrame(df_freq.groupby('Low Income').sum()['hhsize']/df.groupby('Low Income').sum()['hhsize'])

In [ ]:
#df_minority = pd.DataFrame(df_freq.groupby('Minority').sum()['hhsize']/df.groupby('Minority').sum()['hhsize'])

In [ ]:
#reg_share = df_freq.sum()['hhsize']/df.sum()['hhsize']

In [ ]:
#pd.options.display.float_format = '{:.2f}'.format

In [ ]:
#_df = pd.DataFrame([reg_share,df_minority.loc[1].values[0],df_lowinc.loc[1].values[0]])
#_df.index = ['Region','People of Color','People of Low Income']
#_df.columns = ['Percent with Access to Frequent Transit Service']
#_df['Percent with Access to Frequent Transit Service'] = _df['Percent with Access to Frequent Transit Service']*100
#_df

## Percent of People Walking and Biking for Transport

In [11]:
trip_person = pd.merge(trip,person,on=['hhno','pno'], how='left')
person_hh = pd.merge(person,hh[['hhno','hhtaz']],on='hhno',how='left')
# bike_walk_trips = trip_person[trip_person['mode'].isin(['Bike','Walk'])]
# Walk & bike modes, plus transit trips with walk access
bike_walk_trips = trip_person[trip_person['mode'].isin(['Bike','Walk']) | ((trip_person['mode'] == 'Transit') & (trip_person['dorp'] > 0))]

df = bike_walk_trips.groupby(['hhno','pno']).count()
df = df.reset_index()
df = df[['hhno','pno']]
df['bike_walk'] = True

df = pd.merge(person_hh,df,on=['hhno','pno'], how='left')
df['bike_walk'] = df['bike_walk'].fillna(False)

In [13]:
pd.options.display.float_format = '{:,.1%}'.format
reg_active = pd.DataFrame(df.groupby('bike_walk').sum()['psexpfac']/df['psexpfac'].sum()).loc[True]['psexpfac']

In [14]:
df = pd.merge(df, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
df = pd.merge(df, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [15]:
_df = pd.DataFrame(df[df['Low Income'] == 1].groupby(['bike_walk']).count()).reset_index()
low_inc_active = _df[_df['bike_walk'] == True]['psexpfac'].tolist()[0]/_df.sum()['psexpfac']

In [16]:
_df = pd.DataFrame(df[df['Minority'] == 1].groupby(['bike_walk']).count()).reset_index()
minority_active = _df[_df['bike_walk'] == True]['psexpfac'].tolist()[0]/_df.sum()['psexpfac']

In [17]:
_df = pd.DataFrame([reg_active,minority_active,low_inc_active])
_df.index = ['Region','People of Color','People of Low Income']
_df.columns = ['% Walks or Bikes for Transportation']
_df

,% Walks or Bikes for Transportation
Region,34.0%
People of Color,36.1%
People of Low Income,42.5%


### Average Time Spent Walking

In [18]:
pd.options.display.float_format = "{0:.2f}".format
# walk_trips = trip_person[trip_person['mode'] == 'Walk']
# daily_walk_times = walk_trips.groupby(['hhno','pno']).sum()['travtime']

In [19]:
daily_tot_time = pd.DataFrame(trip_person.groupby(['pno','hhno','mode']).sum()['travtime'])
daily_tot_time = daily_tot_time.reset_index()
daily_walk_time = daily_tot_time[daily_tot_time['mode'] == 'Walk']

In [20]:
# Total walk time across region divided by total number of people
reg_walk_time = daily_walk_time['travtime'].sum()/person['psexpfac'].sum()

In [21]:
# Total for low inc and for rgc
df = pd.merge(daily_tot_time,hh[['hhno','hhtaz']],on='hhno', how='left')
df = pd.merge(df,rgc_taz,left_on='hhtaz',right_on='taz',how='left')
df = pd.merge(df,special_taz,left_on='hhtaz',right_on='TAZ',how='left')

In [22]:
low_inc_walk_time = df[df['Low Income'] == 1]['travtime'].sum()/person['psexpfac'].sum()
minority_walk_time = df[df['Minority'] == 1]['travtime'].sum()/person['psexpfac'].sum()

In [23]:
df = pd.DataFrame([reg_walk_time,minority_walk_time,low_inc_walk_time])
df.columns = ['Average Minutes Walking per Day']
df.index = ['Region','People of Color','People of Low Income']
df

,Average Minutes Walking per Day
Region,12.64
People of Color,13.88
People of Low Income,5.73


# Annual Out-of-Pocket Costs

In [24]:
annual_factor = 300
reg_costs = trip_hh[['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor
low_inc_costs = trip_hh[trip_hh['Low Income'] == 1][['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor
minority_costs = trip_hh[trip_hh['Minority'] == 1][['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor

In [25]:
df = pd.DataFrame([reg_costs,minority_costs,low_inc_costs])
df.columns = ['Annual Out-of-Pocket Costs']
df.index = ['Region','People of Color','People of Low Income']

# Out of pocket costs as percentage of household income
hh = pd.merge(hh, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
hh = pd.merge(hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

regional_avg_inc = hh.groupby('hhno').sum()['hhincome'].median()
low_inc_avg_inc = hh[hh['Low Income'] == 1].groupby('hhno').sum()['hhincome'].median()
minority_avg_inc = hh[hh['Minority'] == 1].groupby('hhno').sum()['hhincome'].median()

df_inc = pd.DataFrame([regional_avg_inc,low_inc_avg_inc,minority_avg_inc])
df_inc.columns = ['Median Income']
df_inc.index = ['Region','People of Low Income','People of Color']

df = pd.merge(df,df_inc,left_index=True, right_index=True)
df['% of Income'] = (df['Annual Out-of-Pocket Costs']/df['Median Income'])*100
df

,Annual Out-of-Pocket Costs,Median Income,% of Income
Region,4230.38,65705.00,6.44
People of Low Income,3123.00,47812.00,6.53
People of Color,3895.13,54455.00,7.15


In [26]:
# Percent of income for households making 30,000
df['% Income for $30000 income household'] = (df[['Annual Out-of-Pocket Costs']]/30000)*100
df['% Income for $20000 income household'] = (df[['Annual Out-of-Pocket Costs']]/20000)*100
df

,Annual Out-of-Pocket Costs,Median Income,% of Income,% Income for $30000 income household,% Income for $20000 income household
Region,4230.38,65705.00,6.44,14.10,21.15
People of Low Income,3123.00,47812.00,6.53,10.41,15.61
People of Color,3895.13,54455.00,7.15,12.98,19.48


### Daily VMT per capita

In [27]:
driver_trips = trip[(trip['dorp'] == 1) & (trip['mode'].isin(['SOV','HOV2','HOV3+']))]

In [28]:
person_hh = pd.merge(person,hh[['hhno','hhtaz']],on='hhno',how='left')
person_hh = pd.merge(person_hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')
person_hh = pd.merge(person_hh, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
df = pd.merge(driver_trips,person_hh,on=['hhno','pno'], how='left')

In [29]:
reg_vmt = df['travdist'].sum()/person_hh['psexpfac'].sum()

In [30]:
vmt = pd.DataFrame(df[['Minority','travdist']].groupby('Minority').sum()['travdist'])
# Population totals
pop = person_hh[['Minority','psexpfac']].groupby(['Minority']).sum()
minority_vmt = pd.DataFrame(vmt['travdist']/pop['psexpfac']).loc[1].values[0]

In [31]:
vmt = pd.DataFrame(df[['Low Income','travdist']].groupby('Low Income').sum()['travdist'])
# Population totals
pop = person_hh[['Low Income','psexpfac']].groupby(['Low Income']).sum()
lowinc_vmt = pd.DataFrame(vmt['travdist']/pop['psexpfac']).loc[1].values[0]

In [32]:
_df = pd.DataFrame([reg_vmt,minority_vmt,lowinc_vmt])
_df.columns = ['Average VMT per Capita']
_df.index = ['Region','People of Color','People of Lower Income']
_df

,Average VMT per Capita
Region,14.28
People of Color,12.64
People of Lower Income,10.49


# Average Auto Delay per Capita

In [33]:
trip_auto = trip_hh[trip_hh['mode'].isin(['SOV','HOV2','HOV3+']) & (trip['dorp'] == 1)]
trip_auto['delay'] = trip_auto['travtime'] - trip_auto['sov_ff_time']/100.0

KeyError: 'sov_ff_time'

In [ ]:
# Regional average
reg_delay = (trip_auto['delay'].sum()/person['psexpfac'].sum())*300/60

In [ ]:
low_inc_delay = trip_auto[trip_auto['Low Income'] == 1]['delay'].sum()/person_hh[person_hh['Low Income'] == 1][['psexpfac']].sum()
low_inc_delay = low_inc_delay.values[0]*300/60

In [ ]:
minority_delay = trip_auto[trip_auto['Minority'] == 1]['delay'].sum()/person_hh[person_hh['Minority'] == 1][['psexpfac']].sum()
minority_delay = minority_delay.values[0]*300/60

In [ ]:
df = pd.DataFrame([reg_delay,low_inc_delay,minority_delay])
df.columns = ['Average Annual Vehicle Delay (hours)']
df.index = ['Region','People of Low Income','People of Color']
df